In [1]:
import os, glob, time, random, gc
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from scipy.interpolate import interp1d
from scipy.integrate import trapezoid

# ==========================================
# [0] Experiment settings
# ==========================================
PULSEDB_DIR = "/content/drive/MyDrive/Colab Notebooks/PulseDB/Normal"
SEGMENT_LIMIT = None
PAD_LEN = 200
SEC_PER_SEGMENT = 10.0

BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
WEIGHT_DECAY = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"Using Device: {DEVICE}")

# protocol config (kept, but LOSO will not use these)
N_FOLDS = 5
TIME_GAP_MIN = 5
TEST_DUR_MIN = 5
TRAIN_DUR_MIN = 15
BLOCK_GAP_MIN = 3
VAL_FRAC_IN_TRAIN = 0.20
FILTERING = False

# ==========================================
# [1] Filtering + resample + priors
# ==========================================
def preprocess_ensemble_by_rpeaks(ppg_raw, rpeaks_raw, sbp, dbp, target_len=125, threshold_corr=0.7):
    if not (50 <= sbp <= 250) or not (30 <= dbp <= 160):
        return None

    ppg = ppg_raw.squeeze()
    rpeaks = rpeaks_raw.squeeze()
    rpeaks = np.sort(rpeaks.astype(int))

    beats = []
    for i in range(len(rpeaks) - 1):
        start, end = rpeaks[i], rpeaks[i + 1]
        if start < 0 or end > len(ppg):
            continue
        beat_segment = ppg[start:end]
        if len(beat_segment) < 20:
            continue

        x_old = np.linspace(0, 1, len(beat_segment))
        x_new = np.linspace(0, 1, target_len)
        f_interp = interp1d(x_old, beat_segment, kind='linear', fill_value="extrapolate")
        beats.append(f_interp(x_new))

    if len(beats) < 5:
        return None

    beats = np.array(beats)
    ensemble_avg = np.mean(beats, axis=0)

    e_min, e_max = ensemble_avg.min(), ensemble_avg.max()
    if e_max - e_min > 1e-6:
        ensemble_avg = (ensemble_avg - e_min) / (e_max - e_min)

    correlations = [np.corrcoef(ensemble_avg, b)[0, 1] for b in beats]
    consistent_beats_count = sum(1 for c in correlations if c >= threshold_corr)
    if (consistent_beats_count / len(beats)) < 0.7:
        return None

    return ensemble_avg.astype(np.float32)

def cubic_resample(ppg, target_len=PAD_LEN):
    x_old = np.linspace(0, 1, len(ppg))
    x_new = np.linspace(0, 1, target_len)
    if len(ppg) < 4:
        return np.interp(x_new, x_old, ppg).astype(np.float32)
    try:
        f = interp1d(x_old, ppg, kind="cubic", bounds_error=False, fill_value="extrapolate")
        return f(x_new).astype(np.float32)
    except Exception:
        return np.interp(x_new, x_old, ppg).astype(np.float32)

def extract_multiscale_morph_features(ppg_01):
    scales = [100, 150, 200, 250]
    all_features = []
    for scale in scales:
        x = cubic_resample(ppg_01, scale)

        peak_idx = int(np.argmax(x))
        end_idx = scale - 1

        vp = float(x[peak_idx])
        vt = float(x[end_idx])
        dv = vp - vt
        vm = float(np.mean(x))
        std_val = float(np.std(x))

        tvp = peak_idx / scale

        diff = np.diff(x)
        kmax = float(np.max(diff)) if len(diff) > 0 else 0.0
        tkmax = (int(np.argmax(diff)) / scale) if len(diff) > 0 else 0.0

        amax = float(trapezoid(x[:peak_idx])) if peak_idx > 0 else 0.0

        centered = x - vm
        skew_approx = float(np.mean(centered**3) / (std_val**3)) if std_val > 0 else 0.0
        kurt_approx = float(np.mean(centered**4) / (std_val**4)) if std_val > 0 else 0.0

        all_features.extend([vp, vt, dv, vm, kmax, tkmax, amax, std_val, tvp, skew_approx, kurt_approx])

    return np.array(all_features, dtype=np.float32)

# ==========================================
# [2] Load data
# ==========================================
def load_data_from_mat(mat_path, segment_limit=None, filtering=False):
    segments, priors, targets = [], [], []
    skip_bp, skip_noise = 0, 0

    with h5py.File(mat_path, "r") as f:
        sw = f["Subj_Wins"]
        ppg_refs = sw["PPG_F"][0]
        sbp_refs = sw["SegSBP"][0]
        dbp_refs = sw["SegDBP"][0]

        total = min(len(ppg_refs), segment_limit) if segment_limit else len(ppg_refs)

        if not filtering:
            for i in range(total):
                ppg = f[ppg_refs[i]][()].squeeze().astype(np.float32)
                sbp = float(f[sbp_refs[i]][()][0][0])
                dbp = float(f[dbp_refs[i]][()][0][0])

                segments.append(ppg)
                priors.append(extract_multiscale_morph_features(ppg))
                targets.append([sbp, dbp])

            print(f"✅ (NoFilter) kept: {len(segments)} / {total}")
            return segments, np.stack(priors).astype(np.float32), np.array(targets, dtype=np.float32)

        # filtering=True
        ecg_refs = sw["ECG_RPeaks"][0]
        for i in range(total):
            ppg_raw = f[ppg_refs[i]][()]
            sbp = float(f[sbp_refs[i]][()][0][0])
            dbp = float(f[dbp_refs[i]][()][0][0])
            rpeaks_raw = f[ecg_refs[i]][()]

            processed_ppg = preprocess_ensemble_by_rpeaks(ppg_raw, rpeaks_raw, sbp, dbp)

            if processed_ppg is None:
                if not (50 <= sbp <= 250) or not (30 <= dbp <= 160):
                    skip_bp += 1
                else:
                    skip_noise += 1
                continue

            segments.append(processed_ppg)
            priors.append(extract_multiscale_morph_features(processed_ppg))
            targets.append([sbp, dbp])

    print(f"✅ (Filter) kept: {len(segments)} / {total}")
    print(f"❌ excluded: (bp_range={skip_bp}, noise/quality={skip_noise})")
    return segments, np.stack(priors).astype(np.float32), np.array(targets, dtype=np.float32)

# ==========================================
# [3] Dataset
# ==========================================
class PPGDatasetRawY(Dataset):
    def __init__(self, segments, priors, targets_mmHg):
        self.segments = segments
        self.priors = priors
        self.targets = targets_mmHg

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        x = cubic_resample(self.segments[idx], PAD_LEN)
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
        p = torch.tensor(self.priors[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, p, y

# ==========================================
# [4] Model
# ==========================================
class MorphCNNRegressor(nn.Module):
    def __init__(self, prior_dim=44):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, 5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, 5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(128, 256, 3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )

        self.fc_prior = nn.Sequential(
            nn.Linear(prior_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )

        self.fc_out = nn.Sequential(
            nn.Linear(256 + 256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x, prior):
        feat = self.cnn(x).squeeze(-1)
        pfeat = self.fc_prior(prior)
        return self.fc_out(torch.cat([feat, pfeat], dim=1))

# ==========================================
# [5] Train-only label scaler
# ==========================================
class LabelScaler2D:
    def __init__(self, mode="minmax", eps=1e-6):
        assert mode in ["minmax", "zscore"]
        self.mode = mode
        self.eps = eps
        self.fitted = False

    def fit(self, y_train_mmHg: np.ndarray):
        y = np.asarray(y_train_mmHg, dtype=np.float32)
        if self.mode == "minmax":
            self.y_min = y.min(axis=0)
            self.y_max = y.max(axis=0)
        else:
            self.y_mean = y.mean(axis=0)
            self.y_std = y.std(axis=0)
        self.fitted = True
        return self

    def transform(self, y_mmHg: torch.Tensor) -> torch.Tensor:
        assert self.fitted
        if self.mode == "minmax":
            y_min = torch.tensor(self.y_min, device=y_mmHg.device, dtype=y_mmHg.dtype)
            y_max = torch.tensor(self.y_max, device=y_mmHg.device, dtype=y_mmHg.dtype)
            return (y_mmHg - y_min) / (y_max - y_min + self.eps)
        else:
            y_mean = torch.tensor(self.y_mean, device=y_mmHg.device, dtype=y_mmHg.dtype)
            y_std = torch.tensor(self.y_std, device=y_mmHg.device, dtype=y_mmHg.dtype)
            return (y_mmHg - y_mean) / (y_std + self.eps)

    def inverse(self, y_scaled: torch.Tensor) -> torch.Tensor:
        assert self.fitted
        if self.mode == "minmax":
            y_min = torch.tensor(self.y_min, device=y_scaled.device, dtype=y_scaled.dtype)
            y_max = torch.tensor(self.y_max, device=y_scaled.device, dtype=y_scaled.dtype)
            return y_scaled * (y_max - y_min + self.eps) + y_min
        else:
            y_mean = torch.tensor(self.y_mean, device=y_scaled.device, dtype=y_scaled.dtype)
            y_std = torch.tensor(self.y_std, device=y_scaled.device, dtype=y_scaled.dtype)
            return y_scaled * (y_std + self.eps) + y_mean

# ==========================================
# [6] Train / Eval
# ==========================================
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def train_one_model(train_loader, val_loader, scaler: LabelScaler2D):
    model = MorphCNNRegressor(prior_dim=44).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    criterion = nn.MSELoss()

    best_val = float("inf")
    best_state = None

    for epoch in range(1, EPOCHS + 1):
        # -------------------
        # Train
        # -------------------
        model.train()
        train_losses = []
        for x, p, y_mmHg in train_loader:
            x, p, y_mmHg = x.to(DEVICE), p.to(DEVICE), y_mmHg.to(DEVICE)
            y = scaler.transform(y_mmHg)

            pred = model(x, p)
            loss = criterion(pred, y)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_losses.append(float(loss.item()))

        avg_train = float(np.mean(train_losses)) if len(train_losses) else float("inf")

        # -------------------
        # Val
        # -------------------
        model.eval()
        val_losses = []
        with torch.no_grad():
            for x, p, y_mmHg in val_loader:
                x, p, y_mmHg = x.to(DEVICE), p.to(DEVICE), y_mmHg.to(DEVICE)
                y = scaler.transform(y_mmHg)

                pred = model(x, p)
                val_losses.append(float(criterion(pred, y).item()))
        avg_val = float(np.mean(val_losses)) if len(val_losses) else float("inf")

        # ✅ 25 epoch마다 로그 출력 (+ 첫 epoch, 마지막 epoch도 출력)
        if (epoch == 1) or (epoch % 25 == 0) or (epoch == EPOCHS):
            print(f"    [ep {epoch:03d}/{EPOCHS}] train={avg_train:.6f} | val={avg_val:.6f}")

        # Best checkpoint by val loss
        if avg_val < best_val:
            best_val = avg_val
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    if best_state is not None:
        model.load_state_dict(best_state)
    return model


def eval_mae_sd_mmHg(model, loader, scaler: LabelScaler2D):
    model.eval()
    errs = []
    with torch.no_grad():
        for x, p, y_mmHg in loader:
            x, p, y_mmHg = x.to(DEVICE), p.to(DEVICE), y_mmHg.to(DEVICE)
            pred_scaled = model(x, p)
            pred_mmHg = scaler.inverse(pred_scaled)
            err = (pred_mmHg - y_mmHg).detach().cpu().numpy()
            errs.append(err)

    if len(errs) == 0:
        return dict(mae_sbp=np.nan, sd_sbp=np.nan, mae_dbp=np.nan, sd_dbp=np.nan, n=0)

    E = np.concatenate(errs, axis=0)
    e_sbp, e_dbp = E[:, 0], E[:, 1]
    return dict(
        mae_sbp=float(np.mean(np.abs(e_sbp))),
        sd_sbp=float(np.std(e_sbp, ddof=0)),
        mae_dbp=float(np.mean(np.abs(e_dbp))),
        sd_dbp=float(np.std(e_dbp, ddof=0)),
        n=int(E.shape[0])
    )

# ==========================================
# [7] LOSO helpers
# ==========================================
def build_patient_dataset(mat_path: str, filtering: bool):
    segments, priors, targets_mmHg = load_data_from_mat(mat_path, segment_limit=SEGMENT_LIMIT, filtering=filtering)
    ds = PPGDatasetRawY(segments, priors, targets_mmHg)
    return ds, targets_mmHg

def concat_datasets(datasets):
    # torch.utils.data.ConcatDataset exists, but we also need y targets for scaler fit
    from torch.utils.data import ConcatDataset
    return ConcatDataset(datasets)

def collect_targets_from_concat(targets_list):
    # targets_list: list of np arrays (Ni,2)
    return np.concatenate(targets_list, axis=0).astype(np.float32)

def summarize_loso_fold_stats(valid_stats):
    # valid_stats is list of dicts per held-out patient
    mae_sbp = np.array([v["mae_sbp"] for v in valid_stats], dtype=np.float32)
    mae_dbp = np.array([v["mae_dbp"] for v in valid_stats], dtype=np.float32)
    sd_sbp  = np.array([v["sd_sbp"]  for v in valid_stats], dtype=np.float32)
    sd_dbp  = np.array([v["sd_dbp"]  for v in valid_stats], dtype=np.float32)

    return {
        "avg_mae_sbp": float(mae_sbp.mean()),
        "avg_mae_dbp": float(mae_dbp.mean()),
        "avg_sd_sbp":  float(sd_sbp.mean()),
        "avg_sd_dbp":  float(sd_dbp.mean()),
        "worst_mae_sbp": float(mae_sbp.max()),
        "worst_mae_dbp": float(mae_dbp.max()),
        "std_mae_sbp": float(mae_sbp.std(ddof=0)),
        "std_mae_dbp": float(mae_dbp.std(ddof=0)),
        "valid_folds": int(len(valid_stats)),
    }

# ==========================================
# [8] Cross-subject LOSO (10 patients)
# ==========================================
def run_cross_subject_loso(pulsedb_dir: str, filtering: bool, max_subjects=10):
    set_seed(SEED)

    patient_files = sorted(glob.glob(os.path.join(pulsedb_dir, "p*.mat")))
    if len(patient_files) == 0:
        raise FileNotFoundError(f"No .mat files found under: {pulsedb_dir}")

    # use first 10 (or less if fewer exist)
    patient_files = patient_files[:max_subjects]
    print("\n" + "#" * 80)
    print(f"[CROSS-SUBJECT LOSO] subjects={len(patient_files)} | filtering={filtering}")
    print(f"DIR: {pulsedb_dir}")
    print("#" * 80)

    # pre-load each patient's dataset once (so we don't repeatedly read .mat)
    per_ds = []
    per_y  = []
    per_n  = []
    for fp in patient_files:
        print("\n" + "-" * 80)
        print(f"[LOAD] {os.path.basename(fp)}")
        ds_i, y_i = build_patient_dataset(fp, filtering=filtering)
        per_ds.append(ds_i)
        per_y.append(y_i)
        per_n.append(len(ds_i))
        print(f"  segments: {len(ds_i)}")

    fold_stats = []
    skipped = []

    t0_all = time.time()

    for test_idx, test_fp in enumerate(patient_files):
        test_name = os.path.basename(test_fp)
        print("\n" + "=" * 80)
        print(f"[LOSO] Held-out TEST = {test_name} ({test_idx+1}/{len(patient_files)})")
        print("=" * 80)

        # train = all except test
        train_ds_list = [per_ds[i] for i in range(len(patient_files)) if i != test_idx]
        train_y_list  = [per_y[i]  for i in range(len(patient_files)) if i != test_idx]
        test_ds       = per_ds[test_idx]

        if len(test_ds) < 1:
            skipped.append((test_name, "empty test subject"))
            continue

        train_concat = concat_datasets(train_ds_list)
        y_train_all  = collect_targets_from_concat(train_y_list)

        n_total = len(train_concat)
        n_val = max(1, int(n_total * VAL_FRAC_IN_TRAIN))
        if n_total - n_val < 1:
            skipped.append((test_name, "train too small after val split"))
            continue

        # IMPORTANT: we split train/val by index on concatenated dataset (no shuffling here)
        train_indices = list(range(0, n_total - n_val))
        val_indices   = list(range(n_total - n_val, n_total))
        test_indices  = list(range(0, len(test_ds)))

        # scaler fit on TRAIN ONLY (exclude val)
        y_train_only = y_train_all[np.array(train_indices)]
        scaler = LabelScaler2D(mode="minmax", eps=1e-6).fit(y_train_only)

        train_loader = DataLoader(Subset(train_concat, train_indices), batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(Subset(train_concat, val_indices),   batch_size=BATCH_SIZE, shuffle=False)
        test_loader  = DataLoader(Subset(test_ds, test_indices),       batch_size=BATCH_SIZE, shuffle=False)

        t0 = time.time()
        try:
            model = train_one_model(train_loader, val_loader, scaler)
            stat = eval_mae_sd_mmHg(model, test_loader, scaler)
            elapsed = time.time() - t0

            stat.update({
                "test_subject": test_name,
                "train_subjects": len(patient_files) - 1,
                "train_n": len(train_indices),
                "val_n": len(val_indices),
                "test_n": len(test_indices),
                "elapsed_s": float(elapsed),
            })
            fold_stats.append(stat)

            print(f"  Train/Val/Test sizes: {len(train_indices)}/{len(val_indices)}/{len(test_indices)}")
            print(f"  SBP: MAE={stat['mae_sbp']:.4f} | SD={stat['sd_sbp']:.4f}")
            print(f"  DBP: MAE={stat['mae_dbp']:.4f} | SD={stat['sd_dbp']:.4f}")
            print(f"  elapsed: {elapsed:.1f}s")

        except Exception as e:
            reason = f"{type(e).__name__}: {e}"
            print(f"[SKIP FOLD] {test_name} | reason: {reason}")
            skipped.append((test_name, reason))

        del model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    valid = [fs for fs in fold_stats if fs is not None and np.isfinite(fs["mae_sbp"])]
    print("\n" + "#" * 80)
    print(f"[LOSO SUMMARY] valid={len(valid)}/{len(patient_files)} | filtering={filtering}")
    print("#" * 80)

    if len(valid) == 0:
        print("No valid folds.")
        return fold_stats, skipped

    summ = summarize_loso_fold_stats(valid)
    print(f"ValidFolds: {summ['valid_folds']}/{len(patient_files)}")
    print(f"Avg   SBP: MAE={summ['avg_mae_sbp']:.4f} | SD={summ['avg_sd_sbp']:.4f}")
    print(f"Avg   DBP: MAE={summ['avg_mae_dbp']:.4f} | SD={summ['avg_sd_dbp']:.4f}")
    print(f"Worst SBP: MAE={summ['worst_mae_sbp']:.4f} | StdAcrossFolds(MAE)={summ['std_mae_sbp']:.4f}")
    print(f"Worst DBP: MAE={summ['worst_mae_dbp']:.4f} | StdAcrossFolds(MAE)={summ['std_mae_dbp']:.4f}")

    if skipped:
        print("\n" + "#" * 80)
        print("[SKIPPED FOLDS]")
        print("#" * 80)
        for p, r in skipped:
            print(f"- {p}: {r}")

    print(f"\n[ALL DONE] Total elapsed: {time.time() - t0_all:.1f}s")
    return fold_stats, skipped

# ==========================================
# [MAIN]
# ==========================================
if __name__ == "__main__":
    run_cross_subject_loso(PULSEDB_DIR, filtering=FILTERING, max_subjects=10)


Using Device: cuda

################################################################################
[CROSS-SUBJECT LOSO] subjects=10 | filtering=False
DIR: /content/drive/MyDrive/Colab Notebooks/PulseDB/Normal
################################################################################

--------------------------------------------------------------------------------
[LOAD] p004679.mat
✅ (NoFilter) kept: 1862 / 1862
  segments: 1862

--------------------------------------------------------------------------------
[LOAD] p017795.mat
✅ (NoFilter) kept: 1726 / 1726
  segments: 1726

--------------------------------------------------------------------------------
[LOAD] p027884.mat
✅ (NoFilter) kept: 1802 / 1802
  segments: 1802

--------------------------------------------------------------------------------
[LOAD] p030670 (1).mat
✅ (NoFilter) kept: 1764 / 1764
  segments: 1764

--------------------------------------------------------------------------------
[LOAD] p043774.mat
✅ (NoFi

UnboundLocalError: cannot access local variable 'model' where it is not associated with a value